Not really nodes. You need to understand that they are exec nodes, can modify messages and decision nodes can only return a decision on where to go.
Make it unnecessarily slow since you need to hop from one state to another.

In [ ]:
# TODO: test that ps3.get_pdf_text(asset_url) is working as expected.


In [ ]:
# !pip uninstall -y foresight
# !pip install git+https://github.com/SAGE-3/next.git@dev#subdirectory=foresight
# !pip install langchain_openai
# !pip install langchain

In [ ]:
from langchain.globals import set_debug
set_debug(True)


In [ ]:
from langgraph.prebuilt import tools_condition

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langgraph.prebuilt import ToolNode

import sys
sys.path.append("/Users/mahdi/Documents/GitHub/next/foresight/")
# from src.config import config as conf, prod_type
import copy
from foresight.Sage3Sugar.pysage3 import PySage3
from src.prompts.prompt_loader import PromptLoader 
from langchain_openai import ChatOpenAI
keys_to_remove = ['_id', '_createdAt', '_createdBy', '_updatedAt', '_updatedBy',
    'touched', 'executeInfo', 'path', 'raised', 
     'roomId', 'boardId', 'lock', 'dragging', 'pinned', 'rotation'
]

PROMPTS_DIR = "src/prompts/"

In [ ]:

# app_id = '4a29b8bf-9e72-4366-8946-d808e2e3d465'

In [ ]:
prod_type= 'local'
conf = {'local': {'seer_server': 'http://127.0.0.1:5002',
  'jupyter_server': 'http://localhost:8888',
  'redis_server': 'localhost',
  'web_server': 'http://localhost:3333',
  'ws_server': 'ws://localhost:3333'}}

# app_id = '4a29b8bf-9e72-4366-8946-d808e2e3d465'
ps3 = PySage3(conf, prod_type)
# cb = ps3.rooms[room_id].boards[board_id]

In [ ]:
ps3.get_apps(room_id, board_id, add_tags=True)

In [ ]:
ps3.get_apps(room_id, board_id, add_tags=True, filter_tags= keys_to_remove)

In [ ]:
llm_4o = ChatOpenAI(temperature=0.1, model="gpt-4o")
# llm_4o.invoke("hello")

In [ ]:
# example_prompt = PromptLoader.create_prompt(PROMPTS_DIR, "few_shot", "reframe_select_apps")
# q = 'Can you delete all the red stickies with a greeting or hello'
# reframe_select_chain =  example_prompt | llm_4o
# reframe_select_chain.invoke({"query":q})

In [ ]:
# example_prompt = PromptLoader.create_prompt(PROMPTS_DIR, "few_shot", "select_apps")

# q = 'Can you delete all the red stickies with a greeting'
# all_apps = ps3.get_apps(room_id, board_id, add_tags=True, filter_tags= keys_to_remove)
# prompt_text = example_prompt.format(originalItems=all_apps, query=q)
# prompt_text [0:100]+ " ...."


In [ ]:
# (example_prompt | llm_4o).invoke({"originalItems":all_apps, "query":q})

In [ ]:
pwd

In [ ]:
# create_app_prompt = PromptLoader.create_prompt(PROMPTS_DIR, "zero_shot", "create_app")

# q = 'Can you create a new Stickie that says "here you go"'
# all_apps = ps3.get_apps(room_id, board_id, add_tags=True, filter_tags= keys_to_remove)
# prompt_text = create_app_prompt.format(originalItems=all_apps, query=q)
# print(prompt_text [0:100]+ " ....")


In [ ]:
from src.tools.wall_tools import CreateAppTool
tools = {"CreateAppTool": CreateAppTool(ps3)}
llm_chain = llm_4o.bind_tools([tools["CreateAppTool"]])

In [ ]:
resp = llm_chain.invoke("create a blue stickie with the text Hello in French")
resp

In [ ]:
resp.tool_calls

In [ ]:
resp.tool_calls[0]['args']['smartbit']

In [ ]:
from src.smartbits import SmartBit
sb = SmartBit(**resp.tool_calls[0]['args']['smartbit'])
sb


In [ ]:
data = resp.tool_calls[0]['args']['smartbit']['data']
state = resp.tool_calls[0]['args']['smartbit']['state']
#SmartBit(data=data, state=state)

In [ ]:
tool_name = resp.tool_calls[0]["name"]
tool = tools[tool_name]
tool.invoke({"smartbit":  sb})

In [ ]:
from src.tools.wall_tools import DeleteAppTool
tools = {"DeleteAppTool": DeleteAppTool(ps3)}
llm = llm_4o.bind_tools([tools["DeleteAppTool"]])

In [ ]:
current_apps = ps3.get_apps(room_id, board_id, add_tags=True, filter_tags= keys_to_remove)
current_apps

In [ ]:
delete_prompt = PromptLoader.create_prompt(PROMPTS_DIR, "zero_shot", "delete_app")
delete_prompt

In [ ]:
query = "Delete all the stickies with the text hellow world"
resp  = (delete_prompt | llm).invoke({"originalItems":current_apps, "query": query})
resp

In [ ]:
resp.tool_calls

In [ ]:
tool_name = resp.tool_calls[0]['name']
app_id = resp.tool_calls[0]['args']['app_id']

In [ ]:
tools[tool_name].invoke({"app_id": app_id})

In [ ]:
ps3.get_apps(add_tags=True)

In [ ]:
current_apps = ps3.get_apps(room_id, board_id, add_tags=True, filter_tags= keys_to_remove)
current_apps

In [ ]:
query = "Delete all the mccormick recsys pdf file"
resp  = (delete_prompt | llm).invoke({"originalItems":current_apps, "query": query})
resp

In [ ]:
resp.tool_calls

In [ ]:
ps3.get_apps(filter_tags= keys_to_remove_2)

In [ ]:
all_apps = ps3.get_apps(room_id, board_id, add_tags=True, filter_tags= keys_to_remove)
all_apps


In [ ]:
ps3.get_apps_text(all_apps)

In [ ]:
all_apps

In [ ]:
app = all_apps['ca50b398-86d2-4518-8b67-6a426907d61a']
app

In [ ]:
app_type = app.get('data', {}).get('type', None)
app_type

In [ ]:
text = ""
text += app.get('data', {}).get('state', {}).get('text', "")+"\n"
text

In [ ]:
app

In [ ]:
asset_id = app.get('data', {}).get('state', {}).get('assetid')
asset_id

In [ ]:
ps3.list_assets(asset_id=asset_id)

In [ ]:
ps3.get_app('ec8abf06-eff0-42de-93f5-e8251bbce301')

In [ ]:
ps3.list_assets(asset_id='a202367e-75f2-4cb7-be20-d008c9c77aa3')

In [ ]:
ps3.s3_comm.get_pdf_text('dist/apps/homebase/assets/a51fb128-4832-465a-a959-d5ac7c56a578.pdf', pages=[0])

In [ ]:
ls src/chains/

In [ ]:
import importlib
import src
# importlib.reload(src.chains.chain_factory)
# importlib.reload(src.tools.wall_tools)
from src.chains.chain_factory import ChainFactory
from langchain_core.output_parsers import StrOutputParser

In [ ]:
# cf = ChainFactory()
# joke_chain = cf.create_chain("sumarize", "zero_shot/joke", llm_4o, StrOutputParser)
# joke_chain.invoke({"topic": "ice cream"})

In [ ]:
cf.create_chain("summarize_text", "zero_shot/summarize_text", llm_4o, StrOutputParser)

In [ ]:
text = ps3.s3_comm.get_pdf_text('dist/apps/homebase/assets/92d09434-3607-4dd8-9978-9111190b1873.pdf', pages=[0])
cf.chains['summarize_text'].invoke({"query": text})

In [ ]:
from src.prompts.prompt_loader import PromptLoader
wall_agent_prompt = PromptLoader.create_prompt("src/prompts/", "zero_shot", "wall_agent")


In [ ]:
from src.tools.wall_tools import CreateAppTool, DeleteAppTool, SummarizeAppsTool, CompleteOrEscalate
wall_agent_safe_tools = [CreateAppTool(ps3), SummarizeAppsTool(ps3)]
wall_agent_sensitive_tools = [DeleteAppTool(ps3)]
wall_agent_tools = wall_agent_safe_tools + wall_agent_sensitive_tools
wall_agent_runnable = wall_agent_prompt | llm_4o.bind_tools(
    wall_agent_tools + [CompleteOrEscalate]
)

In [ ]:
from langchain.globals import set_debug
set_debug(False)

# wall_agent_runnable.invoke({"current_apps" :"abc", "messages": [("human", "Query: create a stickie with the text hello world")]})

In [ ]:
# handle_tool_error is now in utils
# create_tool_node_with_fallback is now in utils
# Assistant is now oin Utils


def wall_agent_routes(
    state: State,
) -> Literal[
    "wall_agent_safe_tools",
    "wall_agent_sensitive_tools",
    # "complete_or_escalate",
    "__end__",
]:
    last_msg = state["messages"][-1]
    if not last_msg.tool_calls:
        return "__end__"

    tool_calls = state["messages"][-1].tool_calls
    # did_cancel = any(tc["name"] == CompleteOrEscalate.__name__ for tc in tool_calls)
    # if did_cancel:
    #     return "complete_or_escalate"
    
    safe_toolnames = [t.name for t in wall_agent_safe_tools]
    if all(tc["name"] in safe_toolnames for tc in tool_calls):
        return "wall_agent_safe_tools"
    return "wall_agent_sensitive_tools"



In [ ]:
from typing import Annotated, Optional, Literal, Dict
from langchain.pydantic_v1 import BaseModel, Field, UUID4

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.runnables import Runnable, RunnableConfig






In [ ]:
from langgraph.checkpoint.sqlite import SqliteSaver
import uuid
graph_builder.add_node("wall_agent", Assistant(wall_agent_runnable))
graph_builder.add_node("wall_agent_safe_tools", create_tool_node_with_fallback(wall_agent_safe_tools))
graph_builder.add_node("wall_agent_sensitive_tools", create_tool_node_with_fallback(wall_agent_sensitive_tools))

graph_builder.add_conditional_edges("wall_agent", wall_agent_routes)
graph_builder.add_edge("wall_agent_safe_tools", "wall_agent")
graph_builder.add_edge("wall_agent_sensitive_tools", "wall_agent")

graph_builder.set_entry_point("wall_agent")


In [ ]:
graph.invoke({"messages": [("human", "Create a red stickie with the text hellow world")]}, config)

In [ ]:
# resp = graph.nodes['wall_agent'].invoke({"messages": [("human", "Create a red stickie with the text hellow world")], 'current_apps':'abc'})
# resp

In [ ]:
from src.tools.wall_tools import CreateAppTool, DeleteAppTool
resp = llm_4o.bind_tools([CreateAppTool(ps3), DeleteAppTool(ps3)]).invoke("Create three red stickies with the text hellow world and delete all the yellow the stickies on the wall")



In [ ]:
resp.tool_calls

In [ ]:
tn = ToolNode([CreateAppTool(ps3)])

In [ ]:
tn.invoke({"messages": [resp]})

In [ ]:
!pip install langgraph

In [ ]:
from typing import Annotated, Optional, Literal, Dict
from langchain.pydantic_v1 import BaseModel, Field, UUID4

from typing_extensions import TypedDict

from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages


class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]
    user_id: Optional[UUID4]
    current_apps: Dict


graph_builder = StateGraph(State)

def node_1(state: State):
    return {"messages": [("human", "node_1 input")]}

# def node_2(state: State):
#     return {"messages": [("human", "how are you")]}

node_2_p = ChatPromptTemplate.from_messages(
    [   ("human", "node_2 before messages"),  
        ("placeholder", "{messages}"),
     ("human", "node_2 after messages"),  
     
    ]
).partial()

node_2_runnable = node_2_p | llm_4o

graph_builder.add_node("node_1", node_1)
graph_builder.add_node("node_2", Assistant(node_2_runnable))
graph_builder.add_edge("node_1", "node_2")

graph_builder.set_entry_point("node_1")
graph_builder.set_finish_point("node_2")
graph = graph_builder.compile()

In [ ]:
# from IPython.display import Image, display

# try:
#     display(Image(graph.get_graph().draw_mermaid_png()))
# except:
#     # This requires some extra dependencies and is optional
#     pass

In [ ]:
from langchain_core.messages.human import HumanMessage

In [ ]:
s = {'messages': [HumanMessage(content='Hello Input', id='cba94913-770e-4a33-877b-b4ad0e533550'), HumanMessage(content='node_1 input', id='3b7df344-7495-491d-b358-eb6013dd09b3')]}

In [ ]:
s_2 = node_2_runnable.first.invoke(s)

In [ ]:
node_2_runnable.last.invoke(s_2)

In [ ]:
from langchain_core.messages.ai import AIMessage
result = AIMessage(content="It seems like you're trying to outline a sequence or structure involving nodes and messages. However, the context is a bit unclear. Could you please provide more details or clarify what you're trying to achieve? Are you working on a specific project or problem involving nodes and messages?", response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 35, 'total_tokens': 88}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'stop', 'logprobs': None}, id='run-69c51c62-7b79-45dd-b017-0ca38bc20363-0')

In [ ]:
isinstance(result.content, list)

In [ ]:
tools_condition({"messages": [resp]})

In [ ]:
resp.tool_calls

In [ ]:
####

In [ ]:
from src.tools.wall_tools import CreateAppTool, DeleteAppTool, SummarizeAppsTool, CompleteOrEscalate, CreateAppTool
import sys
sys.path.append("/Users/mahdi/Documents/GitHub/next/foresight/")
from foresight.Sage3Sugar.pysage3 import PySage3
from src.prompts.prompt_loader import PromptLoader
from langgraph.graph import StateGraph
from langgraph.checkpoint.sqlite import SqliteSaver
from src.utils.utils import Assistant, State, create_tool_node_with_fallback
from langchain_openai import ChatOpenAI


import uuid

keys_to_remove = ['_id', '_createdAt', '_createdBy', '_updatedAt', '_updatedBy',
    'touched', 'executeInfo', 'path', 'raised', 
     'roomId', 'boardId', 'lock', 'dragging', 'pinned', 'rotation'
]


### TODO: Check with luc if we have this is config?
def get_config(prod_type='local'):
    conf = {
        'local': {
            'seer_server': 'http://127.0.0.1:5002',
            'jupyter_server': 'http://localhost:8888',
            'redis_server': 'localhost',
            'web_server': 'http://localhost:3333',
            'ws_server': 'ws://localhost:3333'
        }
    }
    return conf[prod_type]

def get_app_config():
    return {
        "configurable": {
            "thread_id": str(uuid.uuid4())
        }
    }

In [ ]:
def initialize_tools(ps3):
    wall_agent_safe_tools = [CreateAppTool(ps3), SummarizeAppsTool(ps3), CreateAppTool(ps3)]
    wall_agent_sensitive_tools = [DeleteAppTool(ps3)]
    wall_agent_tools = wall_agent_safe_tools + wall_agent_sensitive_tools
    return wall_agent_safe_tools, wall_agent_sensitive_tools, wall_agent_tools

# prompts.py

def load_wall_agent_prompt():
    return PromptLoader.create_prompt("src/prompts/", "zero_shot", "wall_agent")

In [ ]:
def wall_agent_routes(state: State, wall_agent_safe_tools):
    last_msg = state["messages"][-1]
    if not last_msg.tool_calls:
        return "__end__"

    tool_calls = state["messages"][-1].tool_calls
    safe_toolnames = [t.name for t in wall_agent_safe_tools]
    if all(tc["name"] in safe_toolnames for tc in tool_calls):
        return "wall_agent_safe_tools"
    return "wall_agent_sensitive_tools"

In [ ]:
def build_graph(wall_agent_runnable, wall_agent_safe_tools, wall_agent_sensitive_tools):
    graph_builder = StateGraph(State)

    graph_builder.add_node("wall_agent", Assistant(wall_agent_runnable))
    graph_builder.add_node("wall_agent_safe_tools", create_tool_node_with_fallback(wall_agent_safe_tools))
    graph_builder.add_node("wall_agent_sensitive_tools", create_tool_node_with_fallback(wall_agent_sensitive_tools))

    graph_builder.add_conditional_edges("wall_agent", lambda state: wall_agent_routes(state, wall_agent_safe_tools))
    graph_builder.add_edge("wall_agent_safe_tools", "wall_agent")
    graph_builder.add_edge("wall_agent_sensitive_tools", "wall_agent")

    graph_builder.set_entry_point("wall_agent")

    memory = SqliteSaver.from_conn_string(":memory:")
    return graph_builder.compile(checkpointer=memory)
    

In [ ]:
# Initialize PySage3
from foresight.Sage3Sugar.pysage3 import PySage3

prod_type= 'local'
conf = {'local': {'seer_server': 'http://127.0.0.1:5002',
  'jupyter_server': 'http://localhost:8888',
  'redis_server': 'localhost',
  'web_server': 'http://localhost:3333',
  'ws_server': 'ws://localhost:3333'}}

room_id = "d6640933-7b9f-4536-a263-02a514ff092a"
board_id = "02fc98df-b4f1-470c-8516-7221c3095de7"


# app_id = '4a29b8bf-9e72-4366-8946-d808e2e3d465'
ps3 = PySage3(conf, prod_type)


In [ ]:
wall_agent_safe_tools, wall_agent_sensitive_tools, wall_agent_tools = initialize_tools(ps3)
wall_agent_safe_tools, wall_agent_sensitive_tools, wall_agent_tools


In [ ]:
wall_agent_prompt = load_wall_agent_prompt()
llm_4o = ChatOpenAI(temperature=0.2, model="gpt-4o")

In [ ]:
wall_agent_runnable = wall_agent_prompt | llm_4o.bind_tools(
    wall_agent_tools + [CompleteOrEscalate], ) 

config = get_app_config()
# Build graph
graph = build_graph(wall_agent_runnable, wall_agent_safe_tools, wall_agent_sensitive_tools)

In [ ]:
config = get_app_config()
current_apps = ps3.get_apps(room_id, board_id, add_tags=True, filter_tags= keys_to_remove)
current_assets = ps3.list_assets()
# graph.invoke({"messages": [("human", "open a the pdf file")], 'current_apps':current_apps, 'current_assets':current_assets}, config)

In [ ]:
resp = graph.nodes["wall_agent"].invoke({"messages": [("human", "Summarize all the stickies on the wall")], 'current_apps':current_apps, 'current_assets':current_assets}, config)

In [ ]:
resp["messages"].tool_calls

In [ ]:
import sqlite3
db = sqlite3.connect("file::memory:")

In [ ]:
dir(graph.checkpointer)

In [ ]:
conn = graph.checkpointer.conn

In [ ]:
query = "SELECT name FROM sqlite_master WHERE type='table';"

In [ ]:
tables = conn.execute(query).fetchall()
print("Tables in the database:", tables)

In [ ]:
x = conn.execute("select * from checkpoints").fetchall()
len(x)

In [ ]:
json.loads(x[0][4].decode('utf-8'))

In [ ]:
ps3.list_assets()

In [ ]:
import sys
sys.path.append("/Users/mahdi/Documents/GitHub/next/foresight/")

from foresight.Sage3Sugar.pysage3 import PySage3

prod_type= 'local'
conf = {'local': {'seer_server': 'http://127.0.0.1:5002',
  'jupyter_server': 'http://localhost:8888',
  'redis_server': 'localhost',
  'web_server': 'http://localhost:3333',
  'ws_server': 'ws://localhost:3333'}}

room_id = "d6640933-7b9f-4536-a263-02a514ff092a"
board_id = "02fc98df-b4f1-470c-8516-7221c3095de7"


# app_id = '4a29b8bf-9e72-4366-8946-d808e2e3d465'
ps3 = PySage3(conf, prod_type)

current_assets = ps3.list_assets()
current_apps = ps3.get_apps(room_id, board_id, add_tags=True, filter_tags= keys_to_remove) 
current_apps = ps3.list_assets()

In [ ]:
current_assets


In [ ]:
from src.prompts.prompt_loader import PromptLoader
from src.tools.wall_tools import CreateAppTool, DeleteAppTool, OpenAssetTool, CompleteOrEscalate
from langchain_openai import ChatOpenAI
llm_4o = ChatOpenAI(temperature=0, model="gpt-4o")
llm = llm_4o.bind_tools([CreateAppTool(ps3), CompleteOrEscalate])
create_app_prompt = PromptLoader.create_prompt(PROMPTS_DIR, "zero_shot", "create_app")
resp  = (create_app_prompt | llm).invoke({"current_apps": current_apps, "query": "Create a red stickie with text of Yo"})


In [ ]:
from langgraph.prebuilt import ToolNode

In [ ]:
ToolNode

In [ ]:
### Final Version

In [ ]:
from langchain.pydantic_v1 import BaseModel
from langchain_openai import ChatOpenAI

llm_4o = ChatOpenAI(temperature=0.1, model="gpt-4o")

class PythonDataScience(BaseModel):
    """ 
    A tool that is able transform a natural query into Python code for data science. The tools knows how to access local files and  and Pandas 
    to execute data science queries. The tools returns the reason it belives this is a data science query.
    """

    reason: str 

    class Config:
        schema_extra = {
            "example": {
                "prompt": "Can you compute the average user's age?",
                "reason": "The user age is a column in the db. I can use pandas to compute the average of that column",
            }
        }

class SAGE3Wall(BaseModel):
    """ 
    A tool that is able transform a natural query into an action on the SAGE3 wall, a sort of large infinite canvas desktop. 
    The Sage3Wall can be used to create and move Stickie, Counter, PDFViewer and ImageViewer and other apps. 
    This tool can also allw the user to interact with text contained in the apps on the wall to answer questions or provide summaries.
    """

    reason: str 

    class Config:
        schema_extra = {
            "example": {
                "prompt": "Can you create a stickie on the wall?",
                "reason": "I am a tool that can maipulare valid apps such as stickies, therefore, I can create stickies on the wall",
            }
        }

class OtherRequest(BaseModel):
    """ 
        A tools is called if the query or prompt form the user is anything but Sage3 or Analytics related
    """

    reason: str 

    class Config:
        schema_extra = {
            "example": {
                "prompt": "What time is it?",
                "reason": "Time is neither a data science of an SAGE3 wall agent taks. So I need to call OtherREquest",
            }
        }



In [1]:
from langchain.globals import set_debug
set_debug(True)

import sys
sys.path.append("/Users/mahdi/Documents/GitHub/next/foresight/")

from foresight.Sage3Sugar.pysage3 import PySage3

prod_type= 'local'
conf = {'local': {'seer_server': 'http://127.0.0.1:5002',
  'jupyter_server': 'http://localhost:8888',
  'redis_server': 'localhost',
  'web_server': 'http://localhost:3333',
  'ws_server': 'ws://localhost:3333'}}

room_id = "d6640933-7b9f-4536-a263-02a514ff092a"
board_id = "02fc98df-b4f1-470c-8516-7221c3095de7"

keys_to_remove = ['_id', '_createdAt', '_createdBy', '_updatedAt', '_updatedBy',
    'touched', 'executeInfo', 'path', 'raised', 
     'roomId', 'boardId', 'lock', 'dragging', 'pinned', 'rotation'
]


# app_id = '4a29b8bf-9e72-4366-8946-d808e2e3d465'
ps3 = PySage3(conf, prod_type)


Configuring ps3 client ... 
Completed configuring Sage3 Client


In [4]:
list(ps3.rooms['d6640933-7b9f-4536-a263-02a514ff092a'].boards['02fc98df-b4f1-470c-8516-7221c3095de7'].smartbits)

[('61535d5f-fd81-43c7-9c56-27f80b60df02',
  Stickie(path='', touched=set(), app_id='61535d5f-fd81-43c7-9c56-27f80b60df02', data=Data(path='.data', touched=set(), position=Position(path='.data.position', touched=set(), x=1500589, y=1500262, z=0), size=Size(path='.data.size', touched=set(), width=400, height=420, depth=0), rotation=Rotation(path='.data.rotation', touched=set(), x=0, y=0, z=0), type='Stickie', raised=True, title='Mahdi Bel-caid', roomId='d6640933-7b9f-4536-a263-02a514ff092a', boardId='02fc98df-b4f1-470c-8516-7221c3095de7', dragging=False, pinned=False), state=StickieState(path='.state', touched=set(), text='test 1', fontSize=36, color='yellow', lock=False, executeInfo=ExecuteInfo(path='.state.executeInfo', touched=set(), executeFunc='', params={}))))]

In [2]:
current_apps = ps3.get_apps(room_id, board_id, add_tags=True, filter_tags= keys_to_remove)
current_assets = ps3.list_assets()


In [ ]:
from src.graph import build_graph, initialize_tools, load_wall_agent_prompt, get_app_config

In [ ]:
wall_agent_safe_tools, wall_agent_sensitive_tools, wall_agent_tools = initialize_tools(ps3)
wall_agent_prompt = load_wall_agent_prompt()


In [ ]:
from langchain_openai import ChatOpenAI
llm_4o = ChatOpenAI(temperature=0.1, model="gpt-4o")
wall_agent_runnable = wall_agent_prompt | llm_4o.bind_tools(wall_agent_tools, tool_choice="any") 

In [ ]:
config = get_app_config()

graph = build_graph(wall_agent_runnable, wall_agent_safe_tools, wall_agent_sensitive_tools)

In [ ]:
current_apps = ps3.get_apps(room_id, board_id, add_tags=True, filter_tags= keys_to_remove)
current_assets = ps3.list_assets()

graph.invoke({"messages": [("human", "Create a stickie that says Ciao")], 'current_apps':current_apps, 'current_assets':current_assets}, config)

In [ ]:
graph.get_state(config).values['messages']

In [ ]:
resp = graph.nodes['wall_agent'].invoke({"messages": [("human", "Create a new stickie that says Bonjour Mundo")], 'current_apps':current_apps, 'current_assets':current_assets}, config)
resp

In [ ]:
resp['messages'].tool_calls

In [ ]:
graph.get_state(config)

In [ ]:
graph.update_state(config, resp, as_node="wall_agent")
current_info = {'current_apps':current_apps, 'current_assets':current_assets}
graph.update_state(config, current_info, as_node="wall_agent")

In [ ]:
graph.get_state(config).values

In [ ]:
graph.get_state(config).next

In [ ]:
resp = graph.nodes['wall_agent_safe_tools'].invoke(graph.get_state(config).values, config)
resp

In [ ]:
resp["messages"][-1]

In [ ]:
graph.update_state(config, resp, as_node="wall_agent_safe_tools")

In [ ]:
graph.get_state(config).values['messages']

In [ ]:
graph.get_state(config).next

In [ ]:
resp = graph.nodes['wall_agent'].invoke(graph.get_state(config).values, config)
resp

In [ ]:
resp['messages'].tool_calls

In [ ]:
graph.update_state(config, resp, as_node="wall_agent")

In [ ]:
graph.get_state(config).next

In [ ]:
resp = graph.nodes['complete'].invoke(graph.get_state(config).values, config)
resp

In [ ]:
graph.update_state(config, resp, as_node="complete")

In [ ]:
graph.get_state(config).next

In [ ]:
# s = graph.get_state(config).values
# last_message = s["messages"][-1]
# last_message

In [ ]:
# from langchain_core.messages import ToolMessage

In [ ]:
        # ToolMessage(
        #     content=last_message.tool_calls[0]["args"]["reason"],
        #     tool_call_id=last_message.tool_calls[0]["id"],
        # )

In [ ]:
graph.invoke({"messages": [("human", "Create a stickie on the wall")], 'current_apps':current_apps, 'current_assets':current_assets}, config)

In [ ]:
resp = graph.nodes['wall_agent'].invoke({"messages": [("human", "Create 5 stickies that say Bonjour in different languages")], 'current_apps':current_apps, 'current_assets':current_assets}, config)
resp

In [ ]:
resp['messages'].tool_calls

In [8]:
s_bits = [x[1] for x in list(ps3.get_smartbits(room_id, board_id))]

In [9]:
s_bits

[Stickie(path='', touched=set(), app_id='61535d5f-fd81-43c7-9c56-27f80b60df02', data=Data(path='.data', touched=set(), position=Position(path='.data.position', touched=set(), x=1500589, y=1500262, z=0), size=Size(path='.data.size', touched=set(), width=400, height=420, depth=0), rotation=Rotation(path='.data.rotation', touched=set(), x=0, y=0, z=0), type='Stickie', raised=False, title='Mahdi Bel-caid', roomId='d6640933-7b9f-4536-a263-02a514ff092a', boardId='02fc98df-b4f1-470c-8516-7221c3095de7', dragging=False, pinned=False), state=StickieState(path='.state', touched=set(), text='test 1', fontSize=36, color='yellow', lock=False, executeInfo=ExecuteInfo(path='.state.executeInfo', touched=set(), executeFunc='', params={}))),
 Stickie(path='', touched=set(), app_id='1cd94bc5-1d2f-4002-9d1a-16a95a3eadf4', data=Data(path='.data', touched=set(), position=Position(path='.data.position', touched=set(), x=1501077, y=1500130, z=0), size=Size(path='.data.size', touched=set(), width=400, height=42

In [11]:
ps3.align_selected_apps(s_bits, align="column", by_dim=2)

Getting ready to adjust positions


In [17]:
from src.tools.wall_tools import RearrangeAppTool

In [18]:
from langchain_openai import ChatOpenAI
llm_4o = ChatOpenAI(temperature=0.1, model="gpt-4o")
llm = llm_4o.bind_tools([RearrangeAppTool()])

In [20]:
from src.prompts.prompt_loader import PromptLoader
prompt = PromptLoader.create_prompt("src/prompts/", "zero_shot", "wall_agent")
prompt

ChatPromptTemplate(input_variables=['current_apps', 'current_assets'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'messages': []}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['current_apps', 'current_assets'], template='You are a basic SAGE3 sub-agent and you can create or delete supported apps (also called widgets) on the wall a sort of infinite canvas desktop. Supported widgets are Stickie, Counter, PDFViewer and ImageViewer. In addition to creating those widgets, you can also access and interact with the text contained in the apps on the wall to answer questions or provide summaries.\nWhen opening an asset, select the asset_id that unambiguously matches the user\'s

In [34]:
resp = ( prompt| llm ).invoke({"messages": [("human", "rearrange the stickies with test 1 and 2 to the top")],
                       "current_apps": current_apps, "current_assets":current_assets})

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "messages": [
    [
      "human",
      "rearrange the stickies with test 1 and 2 to the top"
    ]
  ],
  "current_apps": {
    "1cd94bc5-1d2f-4002-9d1a-16a95a3eadf4": {
      "data": {
        "title": "Mahdi Bel-caid",
        "position": {
          "x": 1501077,
          "y": 1500130,
          "z": 0
        },
        "size": {
          "width": 400,
          "height": 420,
          "depth": 0
        },
        "type": "Stickie",
        "state": {
          "text": "test 2",
          "fontSize": 36,
          "color": "yellow"
        }
      },
      "tags": []
    },
    "61535d5f-fd81-43c7-9c56-27f80b60df02": {
      "data": {
        "title": "Mahdi Bel-caid",
        "position": {
          "x": 1500589,
          "y": 1500262,
          "z": 0
        },
        "size": {
          "width": 400,
          "height": 420,
          "depth": 0
        },
        "type": "Stickie",
        "state

In [35]:
resp.tool_calls

[{'name': 'RearrangeAppTool',
  'args': {'apps_ids': ['61535d5f-fd81-43c7-9c56-27f80b60df02',
    '1cd94bc5-1d2f-4002-9d1a-16a95a3eadf4'],
   'align': 'top'},
  'id': 'call_qsQiczfNCHHQQa6SL5Av9GRu'}]